# RGI scratchpad

In [2]:
from dataclasses import dataclass
from typing import Generic, Any, Optional, Callable

from flax.training import train_state, checkpoints
import optax
import jax.numpy as jnp
import jax

from rgi.core.base import (
    Game,
    GameSerializer,
    TGameState,
    TPlayerState,
    TAction,
    TPlayerId,
    Player,
)
from rgi.games import connect4
from rgi.players.zerozero.zerozero_player import ZeroZeroPlayer
from rgi.players.minimax_player import MinimaxPlayer
from rgi.players.random_player import RandomPlayer
from rgi.players.human_player import HumanPlayer
from rgi.players.zerozero.zerozero_model import (
    StateEmbedder,
    ActionEmbedder,
    ZeroZeroModel,
)
import os
from flax.training import checkpoints

In [3]:
jax.devices()

[CudaDevice(id=0)]

In [6]:
d = jax.devices()[0]
d.device_kind

'NVIDIA GeForce RTX 2070 SUPER'

In [1]:
from rgi.games.connect4 import Connect4
from rgi.players.random_player import RandomPlayer
from rgi.players.human_player import HumanPlayer
from rgi.core.game_runner import run_match

game = Connect4()
players = [HumanPlayer(), RandomPlayer()]

average_rewards = run_match(game, players, num_games=3)
print(f"Average rewards: {average_rewards}")

ImportError: cannot import name 'Connect4' from 'rgi.games.connect4' (/app/rgi/games/connect4/__init__.py)